<center><img src="https://i.imgur.com/hkb7Bq7.png" width="500"></center>


### **Prof. José Manuel Magallanes, PhD**

* Professor, Departamento de Ciencias Sociales, Pontificia Universidad Católica del Perú, [jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe)

* Visiting Professor, Evans School of Public Policy and Governance / Senior Data Science Fellow, eScience Institute, University of Washington, [magajm@uw.edu](mailto:magajm@uw.edu)
_____

_____


# Introduction to Python

### Using Python for Pre Processing

In the session we will see the use of Python to:

1. Collect data as dataframes into Python

2. Preprocess a data frame:
    * Subset data
    * Fix column names
    * Look for non-standar missing values
    * Clean cell values
    * Format data types


3. Merge both tables


4. Prepare a file for further analysis



## 1. Collect data tables into Python

In [ ]:
# Location of data file
linkFile="https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Statistical_Annex_HDI_Table.xlsx"

Reading in a table from a file using pandas, since it is an Excel file, I requires **openpyxl**:

In [ ]:
# available in my computer?
!pip show openpyxl

If not available, please go to Anaconda and install it. Once installed, or if available, continue:

In [ ]:
# choose the right function:
import pandas as pd

hdiFile=pd.read_excel(linkFile)

In [ ]:
storage_options = {'User-Agent': 'Mozilla/5.0'}
hdiFile = pd.read_excel(linkFile, storage_options=storage_options)

## Pre Processing

### Subset data

Just keep what you need; let's check the data head and tail:

In [ ]:
hdiFile.head(10)

In [ ]:
hdiFile.tail()

Above, you do not see countries neither in the first rows nor in the last ones. There are rows at the begining and at the end that are not needed. 

Let's find the row indexes that have the data:

In [ ]:
# the problem is at the tail:
hdiFile.tail(72)

In [ ]:
# is this better?
hdiFile.iloc[7:206,:]

In [ ]:
# subsetting as new DF (copy)
hdiRaw=hdiFile.iloc[7:206,:].copy()
hdiRaw

### Fix column names

Notice that **hdiRaw** do not have the right column names. So we need to recover them from **hdiFile**:

In [ ]:
hdiFile.iloc[[3,4],:]

As you see, the column names are in different positions:

In [ ]:
# and
hdiFile.iloc[4,:2].to_list()

In [ ]:
hdiFile.iloc[3,2:].to_list()

In [ ]:
# save column names 
RealHeaders=hdiFile.iloc[4,:2].to_list()+hdiFile.iloc[3,2:].to_list()
# these are:
RealHeaders

Let's avoid all the "ranks":

In [ ]:
RealHeaders[1:-3]

In [ ]:
# no need for the first one
hdiRaw=hdiRaw.iloc[:,1:-3]
hdiRaw

Then, let's just use the column names needed:

In [ ]:
hdiRaw.columns=RealHeaders[1:-3]
hdiRaw

We still have column names with missing values, let's get rid of those:

In [ ]:
BetterHeaders=hdiRaw.columns.dropna().to_list()
BetterHeaders

In [ ]:
#subsetting again
hdiRaw=hdiRaw.loc[:,BetterHeaders]
hdiRaw.head()

Notice above that the columns:
* Have acronyms in parenthesis.
* Have spaces between words.

Let's see what can be done:

In [ ]:
# bye anything between parenthesis
hdiRaw.columns.str.replace('\(.+\)',"", regex=True)

In [ ]:
# bye anything between parenthesis, bye leading-trailing spaces
hdiRaw.columns.str.replace('\(.+\)',"", regex=True).str.strip()

In [ ]:
# bye anything between parenthesis, bye leading-trailing spaces, title case
hdiRaw.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()

Let's keep this last one for a while:

In [ ]:
#changing column names
hdiRaw.columns=hdiRaw.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()
#so
hdiRaw

Now, it is time to decide how the we want as the shorter column name:

* Same title without spaces:

In [ ]:
hdiRaw.columns.str.replace(" ",'',regex=False)

* Some acronyms: Let's do this step by step.

In [ ]:
# each column names splitted:
[name.split() for name in hdiRaw.columns]

In [ ]:
# first letter of each word
[[word[0] for word in name.split()] for name in hdiRaw.columns]

In [ ]:
# final result
[''.join([word[0] for word in name.split()]) for name in hdiRaw.columns]

Let's keep the first alternative:

In [ ]:
hdiRaw.columns=hdiRaw.columns.str.replace(" ",'',regex=False)

Finally...

In [ ]:
hdiRaw

### Look for non-standar missing values

First check a cell that is full of non-word/non-digit characters:

In [ ]:
# full match!
[hdiRaw.iloc[:,1].str.fullmatch("\W+",na=False)]

The above result is telling you if whether there is or there is not a full match (True/False). You can use that to keep the rows where this is _True_:

In [ ]:
# a quick look...
hdiRaw.iloc[:,1][hdiRaw.iloc[:,1].str.fullmatch("\W+",na=False)]

Let's do this for every column:

In [ ]:
i=0
hdiRaw.iloc[:,i][hdiRaw.iloc[:,i].str.fullmatch("\W+",na=False)]

In [ ]:
i=1
hdiRaw.iloc[:,i][hdiRaw.iloc[:,i].str.fullmatch("\W+",na=False)]

Using **for** loop:

In [ ]:
for i in range(hdiRaw.shape[1]):
    print(hdiRaw.iloc[:,i][hdiRaw.iloc[:,i].str.fullmatch("\W+",na=False)])

Using **try**:

In [ ]:
for i in range(hdiRaw.shape[1]):
    try:
        print(hdiRaw.iloc[:,i][hdiRaw.iloc[:,i].str.fullmatch("\W+",na=False)])
    except:
        pass

This means that the people who created this data set used ".." to represent **missing values**. Let's replace those values:

In [ ]:
hdiRaw.replace(to_replace=[".."],
                value=None,inplace=True)
hdiRaw

### Cleaning cell values

Do the current cell values have issues?

* Keeping complete data

In [ ]:
# with all missing (after the first column)
hdiRaw[hdiRaw.iloc[:,1:].isna().all(axis=1)]

In [ ]:
# with at least one missing (after the first column)
hdiRaw[hdiRaw.iloc[:,1:].isna().any(axis=1)]

In [ ]:
hdiComplete=hdiRaw[~hdiRaw.iloc[:,1:].isna().any(axis=1)].copy()
#
hdiComplete

* Making sure text is clean

In [ ]:
# get rid of leading and trailing spaces in text cells
hdiComplete.Country=hdiComplete.Country.str.strip()

* Checking  numeric columns

In [ ]:
hdiComplete.iloc[:,1:].info()

Numbers have been recognised as **object** type. It might be due to having a non numeric value in one cell, or because it **had** a non-numeric.

In [ ]:
# can you apply math?
hdiRaw.iloc[:,1:].max()

You just need to give format.

In [ ]:
hdiClean=hdiComplete.copy()

### Formatting

From above, we just need to format the numeric columns:

* **Formatting into numeric type**:

In [ ]:
# as easy as:
hdiClean[hdiClean.columns[1:]]=hdiClean.iloc[:,1:].apply(pd.to_numeric)

In [ ]:
#recheck
hdiClean.info()

That was easy!

In [ ]:
hdiFormat=hdiClean.copy()

## Case 3: Integrating and Saving

In [ ]:
demoFormat=pd.read_pickle("demoFormat.pkl")

This should be an easy step:

In [ ]:
hdiFormat.merge(demoFormat)

Notice you have lost countries:

In [ ]:
len(hdiFormat),len(demoFormat)

In [ ]:
onlyHDI=set(hdiFormat.Country)-set(demoFormat.Country)
onlyDEMO=set(demoFormat.Country)-set(hdiFormat.Country)

In [ ]:
onlyHDI

In [ ]:
onlyDEMO

Let's look for good matches, using **thefuzz** (might need to be installed):

In [ ]:
from thefuzz import process as fz

# look for a country in OnlyHDI and return the most similar
[(fz.extractOne(demo, onlyHDI),demo) for demo in sorted(onlyDEMO)]

In [ ]:
[(fz.extractOne(demo, onlyHDI),demo) for demo in sorted(onlyDEMO) \
 if fz.extractOne(demo, onlyHDI)[1]>=90]

Let's recover those above. For that, you need to create a **dictionary of changes**:

In [ ]:
{fz.extractOne(demo, onlyHDI)[0]:demo for demo in sorted(onlyDEMO) \
 if fz.extractOne(demo, onlyHDI)[1]>=90}

In [ ]:
# create the object
changesHDI={fz.extractOne(demo, onlyHDI)[0]:demo \
            for demo in sorted(onlyDEMO) \
            if fz.extractOne(demo, onlyHDI)[1]>=90}

# replace in HDI

hdiFormat.Country.replace(to_replace=changesHDI,inplace=True)

In [ ]:
# how many matches?
hdiFormat.merge(demoFormat)

In [ ]:
# should we do it again?

onlyHDI=set(hdiFormat.Country)-set(demoFormat.Country)
onlyDEMO=set(demoFormat.Country)-set(hdiFormat.Country)

[(fz.extractOne(demo, onlyHDI),demo) for demo in sorted(onlyDEMO)]

In [ ]:
{fz.extractOne(demo, onlyHDI)[0]:demo for demo in sorted(onlyDEMO) \
 if (fz.extractOne(demo, onlyHDI)[1]<86)&\
    (fz.extractOne(demo, onlyHDI)[1]>45)
}

In [ ]:
changesHDI2={fz.extractOne(demo, onlyHDI)[0]:demo for demo in sorted(onlyDEMO) \
 if (fz.extractOne(demo, onlyHDI)[1]<86)&\
    (fz.extractOne(demo, onlyHDI)[1]>45)
}

In [ ]:
#manually
changesHDI2.update({'Korea (Republic of)':'South Korea'})
changesHDI2

In [ ]:
# Now
hdiFormat.Country.replace(to_replace=changesHDI2,inplace=True)

Then,

In [ ]:
hdidem=hdiFormat.merge(demoFormat)
# result:
hdidem

In [ ]:
hdidem.info()

____
____


### <font color="red">Saving File to Disk</font>

#### For future use in Python:

In [ ]:
hdidem.to_pickle("hdidemocia.pkl")
# you will need: DF=pd.read_pickle("hdidemocia.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://...../hdidemocia.pkl"),compression=None)

#### For future  use in R:

In [ ]:
!pip show rpy2

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(hdidem,file="hdidem.RDS")

#In R, you call it with: DF = readRDS("hdidem.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://...../hdidem.RDS")